In [1]:
# Nice notebook environment
%matplotlib inline
from IPython.display import display

# Importing numpy
import numpy as np
from numpy import *

# Importing matplotlib for graphics and fixing the default size of plots
import matplotlib
from matplotlib import rc
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors # para colores en las gráficas
from matplotlib.ticker import AutoMinorLocator

matplotlib.rcParams['mathtext.fontset']='cm'
matplotlib.rcParams.update({'font.size': 18})
plt.rcParams["figure.figsize"] = (10,8)

# In case we need sympy
from sympy import init_printing
init_printing(use_latex=True)

# For working with well stabished parameters
from mendeleev import element
from scipy.constants import physical_constants

import tarfile   # para tratar los archivos .tar
import glob      # para buscar directorios
import re        # para expresiones regulares

In [62]:
import numpy as np
import torch
import torch.nn as nn
import dgl
from dgl.nn import GraphConv, DenseGraphConv, SumPooling, AvgPooling
from dgl.nn.pytorch import Sequential


def graph_potential(n_gc_layers: int = 1,
                    n_fc_layers: int = 1,
                    activation_function: str = 'Tanh',
                    pooling: str = 'add',
                    n_node_features: int = 55,
                    n_edge_features: int = 1,
                    n_neurons: int = None) -> torch.nn.Module: 

    r""" 
    A function that returns a model with the desired number of Graph-Convolutional (n_gc_layers) layers and 
    Fully-Connected (n_fc_layers) linear layers, using the specified non-linear activation layers interspaced
    between them. 

    Args:
       
        n_gc_layers (int): number of Graph-Convolution layers (GraphConv from dgl); default: 1 
        n_fc_layers (int): number of densely-connected -Fully Connected- linear layers (DenseGraphConv); default: 1
        activation_function (char): nonlinear activation layer; can be any activation available in torch; default: Tanh
        pooling (char): the type of pooling of node results; can be 'add', i.e. summing over all nodes (e.g. to 
                return total energy; default) or 'mean', to return energy per atom.
        n_node_features (int): length of node feature vectors; default: 55
        
        
        n_edge_features (int): length of edge feature vectors; currently 1, the distance.
        ATENCION PQ CON LA FUNCION QUE ESTAMOS USANDO NO ES NECESARIO SABER LOS EDGES
        
        n_neurons (int): number of neurons in deep layers (if they exist) in the densely-connected network.  
                       If set to None, it is internally set to n_node_features; default: None

    """


    activation_layer = eval("torch.nn." + activation_function + "()")
    
    if pooling == 'add':    
        
        poolingLayer = SumPooling()
   
    else:
    
        poolingLayer = AvgPooling()

    if n_neurons == None:
        
        n_neurons = n_node_features

    
    # first the Graph-convolutional layers: 
    
    r"""     
        GraphConv(in_feats, out_feats, norm='both', weight=True, bias=True, activation=None, allow_zero_in_degree=False)
    """
    
    conv = GraphConv(in_feats = n_node_features, out_feats = n_node_features, activation = activation_layer)
    # Hacemos la convolución y activación en un sólo comando, luego a layers append se mete directo
    
    model = conv
    print(model(g, g.ndata['fit']))
    
    for n in range(0, n_gc_layers):
    
        model = Sequential(model, conv)
        print(model(g, g.ndata['fit']))
        
    # then the fully-connected layers: 
    
    # Applies a linear transformation to the incoming data: y = xA^T + b
    linear1 = nn.Linear(n_node_features, 1)           # only 1 fully connected layer 
                                                      # (has n_node_features as it is connected to the last conv layer)
    linear2 = nn.Linear(n_node_features, n_neurons)   # conv layer --> fully connected layer
    linear3 = nn.Linear(n_neurons, n_neurons)         # fully connected layer --> fully connected layer
    linear4 = nn.Linear(n_neurons, 1)                 # fully connected layer --> final result
    
    
    if n_fc_layers == 1:
        
        model = Sequential(model, linear1)
        
    else:
        
        model = Sequential(model, linear2)
        print("Hola")
        print(model(g, g.ndata['fit']))
        model = Sequential(model, activation_layer)  ### aqui hay que hacer una activation layer sobre la que hemos hecho la transformacion lineal
        
        for n in range(1, n_fc_layers-1):
            
            model = Sequential(model, linear3)
            
            model= Sequential(model, activation_layer)
            
        # and finally the exit layer
        
        model = Sequential(model, linear4)
    
    # last but not least, the pooling layer

    model = Sequential(model, poolingLayer) # Quitarlo de aquí, en todo caso entre conv layers
    
    return model

In [67]:
#import pdb; pdb.set_trace()

#Creamos un grafo sencillo
g = dgl.graph(([0, 0, 0, 0, 0], [1, 2, 3, 4, 5]), num_nodes=6)
g = dgl.add_self_loop(g)
#print(g.num_nodes())
#print(g.num_edges())

g.ndata['fit'] = torch.ones(g.num_nodes(), 3)
print('n_node_features =', g.ndata['fit'].shape[1])
g.edata['a'] = torch.randn(g.num_edges(), 4)
print('Nodes g:\n',g.ndata['fit'])

activation_function = 'Tanh'
activation_layer = eval("torch.nn." + activation_function + "()")

conv = GraphConv(in_feats = 3, out_feats = 3, activation= activation_layer)  # allow_zero_in_degree = True
conv2 = GraphConv(in_feats = 3, out_feats = 2, activation= activation_layer)
lin1 = nn.Linear(2,1)
res = conv(g, g.ndata['fit'])
print(res)
res = conv(g, res)
print(res)
res = conv2(g, res)
print(res)
res = lin1(res)
print(res)

#model = Sequential(conv, conv)
#model = Sequential(model, conv2)
#res = model(g, g.ndata['fit'])
#print(res)

model2 = graph_potential(n_gc_layers = 3, n_fc_layers = 3, n_node_features = g.ndata['fit'].shape[1], n_edge_features=3)

#n_edge_features = g.edata['a'].shape[1]
res = model2(g, g.ndata['fit'])
print(res)

n_node_features = 3
Nodes g:
 tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[ 0.4950, -0.1238,  0.2440],
        [ 0.8677, -0.2945,  0.5422],
        [ 0.8677, -0.2945,  0.5422],
        [ 0.8677, -0.2945,  0.5422],
        [ 0.8677, -0.2945,  0.5422],
        [ 0.8677, -0.2945,  0.5422]], grad_fn=<TanhBackward0>)
tensor([[0.2420, 0.0119, 0.0166],
        [0.7562, 0.0449, 0.0685],
        [0.7562, 0.0449, 0.0685],
        [0.7562, 0.0449, 0.0685],
        [0.7562, 0.0449, 0.0685],
        [0.7562, 0.0449, 0.0685]], grad_fn=<TanhBackward0>)
tensor([[-0.0102,  0.0409],
        [-0.0686,  0.2354],
        [-0.0686,  0.2354],
        [-0.0686,  0.2354],
        [-0.0686,  0.2354],
        [-0.0686,  0.2354]], grad_fn=<TanhBackward0>)
tensor([[-0.6770],
        [-0.5694],
        [-0.5694],
        [-0.5694],
        [-0.5694],
        [-0.5694]], grad_fn=<AddmmBackward0>)
tensor([[-0.3326,  0.315

TypeError: forward() takes 2 positional arguments but 3 were given